In [1]:
import tensorflow as tf
import os
import numpy as np
from pathlib import Path


In [2]:
saved_model_path=Path("trained_model.h5")

In [3]:
model=tf.keras.models.load_model(saved_model_path
                                 )

In [4]:
convertor=tf.lite.TFLiteConverter.from_keras_model(model)

In [5]:
tflite_model=convertor.convert()

In [6]:
tflite_model_dir = Path("./tflite_model_dir")
tflite_model_dir.mkdir(exist_ok=True, parents=True)

In [7]:
tflite_model_file=tflite_model_dir/'vgg16_model.tflite'
tflite_model_file.write_bytes(tflite_model)

59068504

In [8]:
convertor.optimizations=[tf.lite.Optimize.DEFAULT]
tflite_quant_model = convertor.convert()

tflite_model_quant_file = tflite_model_dir/"vgg16_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

14837664

In [9]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [10]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()


In [11]:
import matplotlib.pyplot as plt
test_img = plt.imread("hymenoptera_data/unzip_data/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")

In [12]:
resize_img=tf.image.resize(test_img,(224,224))

In [13]:
input_data=tf.expand_dims(resize_img,0)

In [14]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

interpreter_quant.set_tensor(input_index, input_data)
interpreter_quant.invoke()
pred = interpreter_quant.get_tensor(output_index)

label_map = {0: 'ants', 1: 'bees'}


argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]


'bees'